### Xavier Normal

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
args_batch_size = 64
args_test_batch_size = 1000
args_epochs = 10
args_lr = 0.01
args_momentum = 0.5
args_no_cuda = False
args_seed = 1
args_log_interval = 100

args_cuda = not args_no_cuda and torch.cuda.is_available()

torch.manual_seed(args_seed)
if args_cuda:
    torch.cuda.manual_seed(args_seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if args_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_test_batch_size, shuffle=True, **kwargs)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        torch.nn.init.xavier_normal(self.conv1.weight)
        self.batch1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        torch.nn.init.xavier_normal(self.conv2.weight)
        self.batch2 = nn.BatchNorm2d(20)
        # self.conv2_drop = nn.Dropout2d(p=p)
        # self.p = p
        self.fc1 = nn.Linear(320, 50)
        torch.nn.init.xavier_normal(self.fc1.weight)
        self.batch3 = nn.BatchNorm1d(50)
        self.fc2 = nn.Linear(50, 10)
        torch.nn.init.xavier_normal(self.fc2.weight)

    def forward(self, x):
        x = F.relu(self.batch1(F.max_pool2d(self.conv1(x), 2)))
        x = F.relu(self.batch2(F.max_pool2d(self.conv2(x), 2)))
        x = x.view(-1, 320)
        x = F.relu(self.batch3(self.fc1(x)))
        # x = F.dropout(x, training=self.training,p=self.p)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net()
if args_cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args_lr, momentum=args_momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args_log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    train_loss = 0
    train_correct = 0
    test_loss = 0
    test_correct = 0
    for data, target in train_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        train_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    for data, target in test_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        test_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    train_loss /= len(train_loader.dataset)
    test_loss /= len(test_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, test_correct, len(test_loader.dataset),
        100. * test_correct / len(test_loader.dataset)))


for epoch in range(1, args_epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.410698
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.044165
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.580784
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.470741
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.353193
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.202666
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.300946
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.118504
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.248026
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.159683

Train set: Average loss: 0.1373, Accuracy: 58077/60000 (97%)
Test set: Average loss: 0.1328, Accuracy: 9687/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.117622
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.106092
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.107413
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.120246
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.113327
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.091362
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.077317
Train Epoch: 2 [44800/60000 (75%)]	Loss:

## Kaiming Normal

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
args_batch_size = 64
args_test_batch_size = 1000
args_epochs = 10
args_lr = 0.01
args_momentum = 0.5
args_no_cuda = False
args_seed = 1
args_log_interval = 100

args_cuda = not args_no_cuda and torch.cuda.is_available()

torch.manual_seed(args_seed)
if args_cuda:
    torch.cuda.manual_seed(args_seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if args_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_test_batch_size, shuffle=True, **kwargs)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        torch.nn.init.kaiming_normal(self.conv1.weight)
        self.batch1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        torch.nn.init.kaiming_normal(self.conv2.weight)
        self.batch2 = nn.BatchNorm2d(20)
        # self.conv2_drop = nn.Dropout2d(p=p)
        # self.p = p
        self.fc1 = nn.Linear(320, 50)
        torch.nn.init.kaiming_normal(self.fc1.weight)
        self.batch3 = nn.BatchNorm1d(50)
        self.fc2 = nn.Linear(50, 10)
        torch.nn.init.kaiming_normal(self.fc2.weight)

    def forward(self, x):
        x = F.relu(self.batch1(F.max_pool2d(self.conv1(x), 2)))
        x = F.relu(self.batch2(F.max_pool2d(self.conv2(x), 2)))
        x = x.view(-1, 320)
        x = F.relu(self.batch3(self.fc1(x)))
        # x = F.dropout(x, training=self.training,p=self.p)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net()
if args_cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args_lr, momentum=args_momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args_log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    train_loss = 0
    train_correct = 0
    test_loss = 0
    test_correct = 0
    for data, target in train_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        train_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    for data, target in test_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        test_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    train_loss /= len(train_loader.dataset)
    test_loss /= len(test_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, test_correct, len(test_loader.dataset),
        100. * test_correct / len(test_loader.dataset)))


for epoch in range(1, args_epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.437573
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.164141
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.716657
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.584041
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.499569
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.269609
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.350062
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.149985
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.296009
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.200488

Train set: Average loss: 0.1751, Accuracy: 57507/60000 (96%)
Test set: Average loss: 0.1658, Accuracy: 9598/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.113085
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.156129
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.135328
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.151429
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.203595
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.091125
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.087755
Train Epoch: 2 [44800/60000 (75%)]	Loss:

## Normal

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
args_batch_size = 64
args_test_batch_size = 1000
args_epochs = 10
args_lr = 0.01
args_momentum = 0.5
args_no_cuda = False
args_seed = 1
args_log_interval = 100

args_cuda = not args_no_cuda and torch.cuda.is_available()

torch.manual_seed(args_seed)
if args_cuda:
    torch.cuda.manual_seed(args_seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if args_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data_mnist', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args_test_batch_size, shuffle=True, **kwargs)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        torch.nn.init.normal(self.conv1.weight)
        self.batch1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        torch.nn.init.normal(self.conv2.weight)
        self.batch2 = nn.BatchNorm2d(20)
        # self.conv2_drop = nn.Dropout2d(p=p)
        # self.p = p
        self.fc1 = nn.Linear(320, 50)
        torch.nn.init.normal(self.fc1.weight)
        self.batch3 = nn.BatchNorm1d(50)
        self.fc2 = nn.Linear(50, 10)
        torch.nn.init.normal(self.fc2.weight)

    def forward(self, x):
        x = F.relu(self.batch1(F.max_pool2d(self.conv1(x), 2)))
        x = F.relu(self.batch2(F.max_pool2d(self.conv2(x), 2)))
        x = x.view(-1, 320)
        x = F.relu(self.batch3(self.fc1(x)))
        # x = F.dropout(x, training=self.training,p=self.p)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net()
if args_cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args_lr, momentum=args_momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args_log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    train_loss = 0
    train_correct = 0
    test_loss = 0
    test_correct = 0
    for data, target in train_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        train_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        train_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    for data, target in test_loader:
        if args_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        test_correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    train_loss /= len(train_loader.dataset)
    test_loss /= len(test_loader.dataset)
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, train_correct, len(train_loader.dataset),
        100. * train_correct / len(train_loader.dataset)))
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, test_correct, len(test_loader.dataset),
        100. * test_correct / len(test_loader.dataset)))


for epoch in range(1, args_epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 5.029402
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.268645
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.526663
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.575976
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.762419
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.167113
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.383495
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.050222
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.382146
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.198918

Train set: Average loss: 0.9679, Accuracy: 41258/60000 (69%)
Test set: Average loss: 0.9546, Accuracy: 6855/10000 (69%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.807331
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.777677
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.971694
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.911108
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.942362
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.821116
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.652338
Train Epoch: 2 [44800/60000 (75%)]	Loss:

## Report

|Weight initialization|Train Performance|Test Performance|
|---|---|---|
|Xavier|59683/60000 (99%)|9902/10000 (99%)|
|Kaiming|59488/60000 (99%)|9870/10000 (99%)|
|Normal|55209/60000 (92%)|9211/10000 (92%)|

* Xavier weight initialization performs better of all the three initizliations that I have tried.
* In general, proper weight initialization helps learn the model faster and gives better performance which can be seen here with Xavier or Kaiming.
* Normal weight initialization give worse performance when compared to defaut random weight initialization.